In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
# we're in insights on a local kernel, but through DB connect we can access data on the DBFS
path = "/FileStore/tables/oil_and_ng_wells_hifld_opendata.csv"
df = spark.read.csv(path, header="true", inferSchema="true")

In [ ]:
# looks like there's about 1.5m rows and 35 columns
print(df.count(), len(df.columns))

In [ ]:
df.show(n=10)

In [ ]:
# my "X" column was inferred as a string - correcting
from pyspark.sql.types import DoubleType

df = df.withColumn("X", df["X"].cast(DoubleType()))

In [ ]:
from pyspark.ml.feature import VectorAssembler

cols = ["X", "Y"]

assembler = VectorAssembler(
  inputCols=cols, 
  outputCol="features", 
  handleInvalid="skip"
)

locations = assembler.transform(df)

In [ ]:
from pyspark.ml.clustering import KMeans

# fit a k-means model using the new "features" column
km = KMeans(k=50)
model = km.fit(locations.select("features")

In [ ]:
clusters = model.transform(locations)

from pyspark.sql.functions import col

active = clusters.select(["prediction", "features", "API", "STATUS"]).filter(col("STATUS") != "NON-ACTIVE WELL")
active.show(n=10)

In [ ]:
# plotting active wells - in Insights, you can click the resulting chart and add it directly to a workbook
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True, rc={"figure.figsize":(12.7,10.27)})

pdf = active.groupBy("STATUS").count().toPandas()
pdf['status'] = ["prod, na", "dev", "active", "unknown", "tx", "smo", "prod", "O&G"] # less verbose labels

sns.catplot(x="status", y="count", hue="status", kind="bar", data=pdf)